# Amazon Customer Segment: Actionable Insights Report


## Introduction
This report analyzes pre-defined customer segments to answer critical business questions. The goal is to provide targeted, data-driven strategies for marketing, customer retention, and operational improvements.


In [ ]:
# --- Setup and Imports ---
%pip install seaborn matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob

# Set plot style
sns.set_theme(style="whitegrid")

# --- Output Directory Setup ---
output_dir = 'output_2'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)



  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.6-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp313-cp313-win_amd64.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached matplotlib-3.10.6-cp313-cp313-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl (226 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.60.1-cp313-cp313-win_amd64.whl (2.3 MB)
Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl (73 kB)
Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl (7.0 MB)
U

In [ ]:
# --- Load Data ---
# Try multiple options and auto-detect if needed
cluster_candidates = [
    'outputs/amazon_customer_clusters.csv',
    'outputs/amazon_customers_annotated.csv',
]

original_candidates = [
    'outputs/Amazon Customer Behavior Survey.csv',
]

# Also scan any csvs in outputs/ as fallback
outputs_csvs = glob.glob('outputs/*.csv')

def pick_cluster_file(candidates, fallbacks):
    for path in candidates:
        if os.path.exists(path):
            return path
    # Fallback: look for required columns
    for path in fallbacks:
        try:
            sample = pd.read_csv(path, nrows=50)
            if 'Cluster' in sample.columns:
                return path
        except Exception:
            continue
    raise FileNotFoundError("Could not find a clustered data CSV in 'outputs/'.")

def pick_original_file(candidates, fallbacks):
    for path in candidates:
        if os.path.exists(path):
            return path
    # Fallback: any CSV is acceptable as long as it has Product_Search_Method
    for path in fallbacks:
        try:
            sample = pd.read_csv(path, nrows=50)
            if 'Product_Search_Method' in sample.columns:
                return path
        except Exception:
            continue
    raise FileNotFoundError("Could not find the original survey CSV in 'outputs/'.")

clustered_data_file = pick_cluster_file(cluster_candidates, outputs_csvs)
original_data_file = pick_original_file(original_candidates, outputs_csvs)

print(f"Using clustered data: {clustered_data_file}")
print(f"Using original data:  {original_data_file}")

# Load
try:
    df = pd.read_csv(clustered_data_file)
    df_original = pd.read_csv(original_data_file)
except FileNotFoundError as e:
    print(f"File not found: {e}")
    raise

# Display preview of main dataframe
df.head()


File not found: [Errno 2] No such file or directory: 'outputs/amazon_customer_clusters.csv'


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/amazon_customer_clusters.csv'

## Analysis 1: Prime Membership Candidates Identifier
**Business Question:** Which non-Prime members are the most valuable targets for a Prime membership marketing campaign?

**Methodology:** We identify customers who are not currently Prime members but exhibit behaviors of a loyal customer (i.e., high purchase frequency).


In [ ]:
# --- Analysis 1: Prime Candidates Function ---
from typing import Tuple

def identify_prime_candidates(data: pd.DataFrame, frequency_threshold: int = 3) -> pd.DataFrame:
    """Return customers who are not Prime members but have high purchase frequency.

    Parameters
    ----------
    data : pd.DataFrame
        Input dataframe containing Membership_Status and Purchase_Frequency columns.
    frequency_threshold : int, default 3
        Minimum purchase frequency to qualify as a candidate.

    Returns
    -------
    pd.DataFrame
        Filtered dataframe of candidate customers.
    """
    required_cols = {"Membership_Status", "Purchase_Frequency"}
    missing = required_cols - set(data.columns)
    if missing:
        raise KeyError(f"Missing required columns for Analysis 1: {missing}")

    candidates = data[(data["Membership_Status"] == "Not a Member") & (data["Purchase_Frequency"] >= frequency_threshold)]
    return candidates.copy()

# Execute Analysis 1
prime_candidates = identify_prime_candidates(df, frequency_threshold=3)
print(f"Prime candidates found: {len(prime_candidates)}")
prime_candidates_path = os.path.join(output_dir, 'prime_candidates.csv')
prime_candidates.to_csv(prime_candidates_path, index=False)
prime_candidates.head()


## Analysis 2: "At-Risk" Churn Scorecard
**Business Question:** Which customer segment is most likely to stop using our service (churn)?

**Methodology:** We create a scorecard for each cluster based on a combination of low shopping satisfaction, high return frequency, and a high number of customer service interactions.


In [ ]:
# --- Analysis 2: At-Risk Scorecard Function ---
def calculate_at_risk_scorecard(data: pd.DataFrame) -> pd.DataFrame:
    """Compute cluster-level risk indicators.

    Returns a dataframe with average Shopping_Satisfaction, Return_Frequency,
    and Customer_Service_Interactions by Cluster.
    """
    required_cols = {"Cluster", "Shopping_Satisfaction", "Return_Frequency", "Customer_Service_Interactions"}
    missing = required_cols - set(data.columns)
    if missing:
        raise KeyError(f"Missing required columns for Analysis 2: {missing}")

    scorecard = (
        data.groupby("Cluster", as_index=False)[
            ["Shopping_Satisfaction", "Return_Frequency", "Customer_Service_Interactions"]
        ]
        .mean()
        .rename(columns={
            "Shopping_Satisfaction": "Avg_Shopping_Satisfaction",
            "Return_Frequency": "Avg_Return_Frequency",
            "Customer_Service_Interactions": "Avg_Customer_Service_Interactions"
        })
    )
    return scorecard

# Execute Analysis 2
at_risk_scorecard = calculate_at_risk_scorecard(df)
print(at_risk_scorecard)
at_risk_path = os.path.join(output_dir, 'at_risk_scorecard.csv')
at_risk_scorecard.to_csv(at_risk_path, index=False)
at_risk_scorecard.head()


## Analysis 3: Targeted Advertising Channel Advisor
**Business Question:** Where should we spend our advertising budget to most effectively reach each customer segment?

**Methodology:** We analyze the most commonly used product search method for each cluster.


In [ ]:
# --- Analysis 3: Ad Advisor Function and Visualization ---
from statistics import mode

def get_top_search_methods(data: pd.DataFrame, original_data: pd.DataFrame) -> pd.DataFrame:
    """Join Product_Search_Method from original data and compute cluster modes.

    Returns a dataframe with the most common Product_Search_Method per Cluster.
    """
    # Determine join key(s). Assume both have a stable unique index if present; else fall back to row order.
    left = data.reset_index(drop=True)
    right = original_data.reset_index(drop=True)

    if "Product_Search_Method" not in right.columns:
        raise KeyError("'Product_Search_Method' not found in original_data")

    merged = left.copy()
    merged["Product_Search_Method"] = right["Product_Search_Method"]

    # Compute mode per cluster; handle multimodal by choosing the first.
    top_methods = (
        merged.groupby("Cluster")["Product_Search_Method"]
        .agg(lambda s: s.mode().iat[0] if not s.mode().empty else None)
        .reset_index()
        .rename(columns={"Product_Search_Method": "Top_Product_Search_Method"})
    )

    return top_methods, merged

# Execute Analysis 3
top_methods, df_with_search = get_top_search_methods(df, df_original)
print(top_methods)

plt.figure(figsize=(10, 6))
sns.countplot(data=df_with_search, x="Product_Search_Method", hue="Cluster")
plt.title("Distribution of Product Search Methods by Cluster")
plt.xlabel("Product Search Method")
plt.ylabel("Count")
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
advertising_plot_path = os.path.join(output_dir, 'advertising_channels.png')
plt.savefig(advertising_plot_path, dpi=300)
plt.close()

top_methods.head()


## Analysis 4: Recommendation Engine Effectiveness Check
**Business Question:** Does using our personalized recommendations correlate with higher customer satisfaction?

**Methodology:** We compare the average shopping satisfaction for customers who use recommendations 'Often' versus those who use them 'Rarely'/'Never'.


In [ ]:
# --- Analysis 4: Recommendation Effectiveness Function and Visualization ---

def check_recommendation_effectiveness(data: pd.DataFrame) -> pd.DataFrame:
    """Compare average Shopping_Satisfaction between high vs low recommendation usage groups.

    Categorize 'Personalized_Recommendation_Frequency' into High Usage vs Low Usage and
    compute mean Shopping_Satisfaction by group.
    """
    required_cols = {"Personalized_Recommendation_Frequency", "Shopping_Satisfaction"}
    missing = required_cols - set(data.columns)
    if missing:
        raise KeyError(f"Missing required columns for Analysis 4: {missing}")

    usage_map = {
        "Often": "High Usage",
        "Frequently": "High Usage",
        "Always": "High Usage",
        "Sometimes": "Low Usage",
        "Rarely": "Low Usage",
        "Never": "Low Usage"
    }

    temp = data.copy()
    temp["Recommendation_Usage"] = temp["Personalized_Recommendation_Frequency"].map(usage_map).fillna("Low Usage")

    comparison = (
        temp.groupby("Recommendation_Usage", as_index=False)["Shopping_Satisfaction"].mean()
        .rename(columns={"Shopping_Satisfaction": "Avg_Shopping_Satisfaction"})
    )

    return comparison, temp

# Execute Analysis 4
rec_effectiveness, df_with_usage = check_recommendation_effectiveness(df)
print(rec_effectiveness)

plt.figure(figsize=(8, 5))
sns.barplot(data=rec_effectiveness, x="Recommendation_Usage", y="Avg_Shopping_Satisfaction", palette="viridis")
plt.title("Shopping Satisfaction by Recommendation Usage")
plt.xlabel("Recommendation Usage")
plt.ylabel("Average Shopping Satisfaction")
plt.tight_layout()
rec_plot_path = os.path.join(output_dir, 'recommendation_effectiveness.png')
plt.savefig(rec_plot_path, dpi=300)
plt.close()

rec_effectiveness


## Analysis 5: Customer Service Demand Forecaster
**Business Question:** Which customer segment requires the most attention from our customer service team?

**Methodology:** We calculate the average number of customer service interactions for each cluster.


In [ ]:
# --- Analysis 5: Service Demand Function and Visualization ---

def forecast_service_demand(data: pd.DataFrame) -> pd.DataFrame:
    """Compute average customer service interactions per cluster."""
    required_cols = {"Cluster", "Customer_Service_Interactions"}
    missing = required_cols - set(data.columns)
    if missing:
        raise KeyError(f"Missing required columns for Analysis 5: {missing}")

    demand = (
        data.groupby("Cluster", as_index=False)["Customer_Service_Interactions"].mean()
        .rename(columns={"Customer_Service_Interactions": "Avg_Customer_Service_Interactions"})
    )
    return demand

# Execute Analysis 5
service_demand = forecast_service_demand(df)
print(service_demand)

plt.figure(figsize=(8, 5))
sns.barplot(data=service_demand, x="Cluster", y="Avg_Customer_Service_Interactions", palette="mako")
plt.title("Forecast: Customer Service Interactions by Cluster")
plt.xlabel("Cluster")
plt.ylabel("Average Customer Service Interactions")
plt.tight_layout()
service_plot_path = os.path.join(output_dir, 'service_demand_forecast.png')
plt.savefig(service_plot_path, dpi=300)
plt.close()

service_demand.head()


## Report Conclusion
The analyses above provide several actionable insights for enhancing business performance.
